# Demo of pre-system Formula and PreSystem classes
By: Magnus Kvåle Helliesen

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pre_system.formula import Formula, Indicator, FInflate, FDeflate, FSum, FSumProd, FMult, FDiv, FJoin
from pre_system.pre_system import PreSystem

## Defining formulae
There are a bunch of different Formula *child*-classes, some of which are put to use below.

In [ ]:
# Let's make a formula that extrapolates x using x1, x2 and x3
xa = Indicator('xa', 'xa', ['x0', 'x1', 'x2'])
xb = Indicator('xb', 'xb', ['x3', 'x4'])

# Let's deflate them with a bunch of weighted price indices
vxa = FDeflate('vxa', xa, ['p0', 'p1', 'p2'], ['w0', 'w1', 'w2'])
vxb = FDeflate('vxb', xb, ['p3', 'p4'], ['w3', 'w4'])

# Finally, let's sum them up
x = FSum('xy', xa, xb)
vx = FSum('vxy', vxa, vxb)

## Looking at the textual representation of the formulae
We haven't any data yet, but we can look at the textual representation of the formulae.

In [ ]:
xa

In [ ]:
xb

In [ ]:
vxa

In [ ]:
vxb

In [ ]:
x

In [ ]:
vx

We can also trace back to any formulae that go into some formula.

In [ ]:
vxa.info()

We can also look at pairwise indicators and weights

In [ ]:
# With trace
vxa.indicators_weights()

In [ ]:
# Without trace
vxa.indicators_weights(trace=False)

## Subjecting formulae to data (evaluating formulae with respect to data)
Let's make some random data and store them in Pandas DataFrames. Importantly, the data *must* be indexed by Pandas PeriodIndices.

In [ ]:
years = 13

In [ ]:
annual_df = pd.DataFrame(
    np.exp(0.02+np.random.normal(0, 0.01, (years, 10)).cumsum(axis=0)),
    columns=[f'x{i}' for i in 'abcdefghij'],
    index=pd.period_range(start='2010', periods=years, freq='a')
)

annual_df.plot(figsize=(15, 2.5))
plt.title('Annual values')
plt.show()

In [ ]:
indicator_df = pd.DataFrame(
    np.exp(0.02+np.random.normal(0, 0.01, (years*12, 10)).cumsum(axis=0)),
    columns=[f'x{i}' for i in range(5)]+[f'p{i}' for i in range(5)],
    index=pd.period_range(start='2010-01', periods=years*12, freq='m')
)

indicator_df.plot(figsize=(15, 2.5))
plt.title('Indicators')
plt.show()

In [ ]:
weight_df = pd.DataFrame(
    10+np.random.normal(0, 1, (years, 5)).cumsum(axis=0),
    columns=[f'w{i}' for i in range(5)],
    index=pd.period_range(start='2010', periods=years, freq='a')
)

weight_df[['w0', 'w1', 'w2']] = weight_df[['w0', 'w1', 'w2']].divide(weight_df[['w0', 'w1', 'w2']].sum(axis=1), axis=0)
weight_df[['w3', 'w4']] = weight_df[['w3', 'w4']].divide(weight_df[['w3', 'w4']].sum(axis=1), axis=0)

weight_df.plot(figsize=(15, 2.5))
plt.title('Weights')
plt.show()

Before we can evaluate the formulae, we need to set a baseyear.

In [ ]:
Formula.baseyear = 2020

In [ ]:
fig = x.evaluate(annual_df, indicator_df, weight_df).plot(figsize=(15, 2.5))
plt.title('x')
plt.show()

In [ ]:
fig = vx.evaluate(annual_df, indicator_df, weight_df).plot(figsize=(15, 2.5))
plt.title('vx')
plt.show()

## Organizing the formulae in the PreSystem class
The PreSystem class is written to contain formulae, and allow the user to easily evaluate them, subject to data (contained by PreSystem).

In [ ]:
# Let's create a PreSystem instance
pre_system = PreSystem('Test PreSystem')
pre_system

The PreSystem is now ready to accept formulae and data.

In [ ]:
pre_system.add_formula(xa)
pre_system.add_formula(xb)
pre_system.add_formula(vxa)
pre_system.add_formula(vxb)
pre_system.add_formula(x)
pre_system.add_formula(vx)
pre_system.info()

In order to evaluate the PreSystem, we need to put data in it, and set the baseyear.

In [ ]:
pre_system.baseyear = 2020
pre_system.annuals_df = annual_df
pre_system.indicators_df = indicator_df
pre_system.weights_df = weight_df
pre_system.info()

Now we can evaluate the PreSystem (horay!).

In [ ]:
pre_system.evaluate